In [35]:
from selenium import webdriver
from selenium.webdriver.support.select import Select
import time 
import pandas as pd
from bs4 import BeautifulSoup

In [36]:
import configparser

In [55]:
# user_id = "" 
# user_pass = ""

In [56]:
#options = webdriver.ChromeOptions()
driver = webdriver.Chrome('./chromedriver')

In [57]:
#driver.set_window_size(1400, 1200)
driver.get("https://www.kaida.co.kr/uat/uia/egovLoginUsr.do")

In [58]:
driver.find_element_by_css_selector('#userId').send_keys(user_id)
driver.find_element_by_css_selector('#userPass').send_keys(user_pass)
driver.execute_script('actionLogin()')

In [59]:
# 2. 사용자 설정통계 페이지 이동

In [60]:
driver.get("https://www.kaida.co.kr/ko/statistics/custom2.do")

In [61]:
# 3. 브랜드, 지역, 구매유형, 연령 데이터 수집

In [62]:
# 구매유형
selector = '#pageBody > article.body_wrap > div > div.select_filter > table > tbody > tr > td:nth-child(8) > div > label'
sales_types = driver.find_elements_by_css_selector(selector)
sales_types = {sales_type.text: sales_type.get_attribute("for") for sales_type in sales_types}
print(len(sales_types), sales_types)

4 {'개인': 'PurchaseType3', '개인-남자': 'PurchaseType1', '개인-여자': 'PurchaseType2', '법인': 'PurchaseType0'}


In [63]:
# 연령
ages = driver.find_elements_by_css_selector(".td_Age > .item_wrap > label")
ages = {age.text: age.get_attribute("for") for age in ages}
print(len(ages), ages)

7 {'~19': 'Age1', '20~29': 'Age2', '30~39': 'Age3', '40~49': 'Age4', '50~59': 'Age5', '60~69': 'Age6', '70~': 'Age7'}


In [64]:
def clear_menu(driver):
    # 나이 체크박스 초기화
    driver.execute_script("$('#td_Age > .item_wrap > label > input').prop('checked', false);")
    # tag 초기화
    driver.execute_script("$('.sel_item button').click();");

In [65]:
def select_menu(driver, sales_type_id, age_id):
    
    # 메뉴 초기화
    clear_menu(driver)
    time.sleep(0.5)
    
    # 구매유형 클릭
    driver.find_element_by_css_selector(f'input#{sales_type_id}').click()
    time.sleep(0.5)
    
    # 연령대 클릭
    try: 
        driver.find_element_by_css_selector(f'input#{age_id}').click()
        time.sleep(0.5)
    except:
        pass

In [66]:
def make_table_df(driver, sales_type, age, year):
    
    # 컬럼 데이터 수집
    columns = driver.find_elements_by_css_selector("table.re_table > thead > tr > th")
    columns = [column.text for column in columns] + [ "sales_type", "age", "year"] 

    # 데이터 프레임 만들기
    df = pd.DataFrame(columns=columns)

    # 데이터 프레임 채우기
    rows = driver.find_elements_by_css_selector("table.re_table > tbody > tr")
    for row in rows:
        cells = row.find_elements_by_css_selector('td,th')
        datas = [cell.text for cell in cells] + [sales_type, age, year]     
        df.loc[len(df)] = datas
    
    return df

In [71]:
dfs = []

for year in range(2020, 2021): #2016, 2022        
    years = Select(driver.find_element_by_xpath("""//*[@id="searchYear"]"""))
    driver.find_element_by_css_selector('#searchDivision > option:nth-child(2)').click()

    for sales_type, sales_type_id in list(sales_types.items()):
        for age, age_id in list(ages.items()):
            print([sales_type, age, year], end=" ")

                    # 메뉴 선택
            select_menu(driver, sales_type_id, age_id)

                    # 검색버튼 클릭
            driver.find_element_by_css_selector('.searchBtn').click()
            time.sleep(0.5)

                    # 데이터 수집해서 데이터 프레임으로 만들기
            df = make_table_df(driver, sales_type, age, year)
            dfs.append(df)

                
# 수집된 데이터 프레임 병합
result_df = pd.concat(dfs)

# index 초기화
result_df.reset_index(drop=True, inplace=True)

# csv 파일로 저장
result_df.to_csv("result1.csv", index=False)
result_df.to_excel('result1.xlsx', index=False)

# 출력
result_df.tail()

['개인', '~19', 2020] ['개인', '20~29', 2020] ['개인', '30~39', 2020] ['개인', '40~49', 2020] ['개인', '50~59', 2020] ['개인', '60~69', 2020] ['개인', '70~', 2020] ['개인-남자', '~19', 2020] ['개인-남자', '20~29', 2020] ['개인-남자', '30~39', 2020] ['개인-남자', '40~49', 2020] ['개인-남자', '50~59', 2020] ['개인-남자', '60~69', 2020] ['개인-남자', '70~', 2020] ['개인-여자', '~19', 2020] ['개인-여자', '20~29', 2020] ['개인-여자', '30~39', 2020] ['개인-여자', '40~49', 2020] ['개인-여자', '50~59', 2020] ['개인-여자', '60~69', 2020] ['개인-여자', '70~', 2020] ['법인', '~19', 2020] ['법인', '20~29', 2020] ['법인', '30~39', 2020] ['법인', '40~49', 2020] ['법인', '50~59', 2020] ['법인', '60~69', 2020] ['법인', '70~', 2020] 

,구분,Jan.,Feb.,Mar.,Apr.,May,Jun.,Jul.,Aug.,Sep.,Oct.,Nov.,Dec.,Total,sales_type,age,year
7647,Volvo - XC60 B5 AWD,0,0,0,28,0,0,0,0,0,0,0,0,28,법인,70~,2020
7648,Volvo - XC60 B6 AWD,0,0,9,16,0,0,0,0,0,0,0,0,25,법인,70~,2020
7649,Volvo - XC60 T8 AWD,78,112,45,16,0,0,0,0,0,0,0,0,251,법인,70~,2020
7650,Volvo - XC90 B6 AWD,0,0,20,22,0,0,0,0,0,0,0,0,42,법인,70~,2020
7651,Volvo - XC90 T8 AWD,49,45,43,27,0,0,0,0,0,0,0,0,164,법인,70~,2020


In [72]:
result_df.to_csv("result2020.csv", index=False)
result_df.to_excel('result2021.xlsx', index=False)

In [90]:
import re

In [91]:
 d = [column for column in result_df["구분"]]

In [ ]:
n = re.compile('(\w+\s?[-]?\w+)\s-|[A-Z]{2}[a-z]{4}')

In [99]:
brand = re.findall('(\w+\s?[-]?\w+)\s-', str(d))
brand 

['Audi',
 'BMW',
 'BMW',
 'Bentley',
 'Honda',
 'Land Rover',
 'MINI',
 'Maserati',
 'Mercedes-Benz',
 'Mercedes-Benz',
 'Porsche',
 'Volkswagen',
 'Audi',
 'Audi',
 'Audi',
 'Audi',
 'Audi',
 'Audi',
 'Audi',
 'Audi',
 'Audi',
 'Audi',
 'Audi',
 'Audi',
 'Audi',
 'Audi',
 'Audi',
 'Audi',
 'Audi',
 'Audi',
 'Audi',
 'Audi',
 'Audi',
 'Audi',
 'Audi',
 'Audi',
 'Audi',
 'Audi',
 'Audi',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'Cadillac',
 'Cadillac',
 'Cadillac',
 'Chevrolet',
 'Chevrolet',
 'Chevrolet',
 'Chevrolet',
 'Chevrolet',
 'Chrysler',
 'Chrysler',
 'Chrysler',
 'Chrysler',
 'Chrysler',
 'Chrysler',
 'Chrys

In [ ]:
re.findall('-\s(\w+\s?\w+\s?\w+\s+\w+)', str(d))

In [54]:
driver.quit()

In [ ]:
result_df

In [86]:
six =  pd.read_excel('result2016.xlsx')
sev = pd.read_excel('result2017copy.xlsx')
eig = pd.read_excel('result2018.xlsx')
nin = pd.read_excel('result2019.xlsx')
tene = pd.read_excel('result1.xlsx')

In [88]:
dd = pd.concat([six, sev, eig, nin, tene])

dd.to_excel("Model_df.xlsx")
dd.to_csv("Model_df.csv", index=False, encoding='utf-8-sig')

In [210]:
data = pd.read_csv("Model_df.csv")
data

,구분,Jan.,Feb.,Mar.,Apr.,May,Jun.,Jul.,Aug.,Sep.,Oct.,Nov.,Dec.,Total,sales_type,age,year
0,Audi - Q2 35 TDI,1,0,0,0,0,0,0,0,0,0,0,0,1,개인,~19,2016
1,BMW - 218d Gran Coupe,0,1,0,0,0,0,0,0,0,0,0,0,1,개인,~19,2016
2,BMW - 520,1,0,0,0,0,0,0,0,0,0,0,0,1,개인,~19,2016
3,Bentley - Continental GT V8,0,1,0,0,0,0,0,0,0,0,0,0,1,개인,~19,2016
4,Honda - CR-V Hybrid,0,0,0,1,0,0,0,0,0,0,0,0,1,개인,~19,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45907,Volvo - XC60 B5 AWD,0,0,0,28,0,0,0,0,0,0,0,0,28,법인,70~,2021
45908,Volvo - XC60 B6 AWD,0,0,9,16,0,0,0,0,0,0,0,0,25,법인,70~,2021
45909,Volvo - XC60 T8 AWD,78,112,45,16,0,0,0,0,0,0,0,0,251,법인,70~,2021
45910,Volvo - XC90 B6 AWD,0,0,20,22,0,0,0,0,0,0,0,0,42,법인,70~,2021


In [211]:
import re

In [212]:
d = [column for column in data["구분"]]

In [213]:
brand = re.findall('(\w+\s?[-]?\w+)\s-', str(d))
data["brand"] = pd.DataFrame(brand)

In [214]:
data['model'] = data['구분'].replace(to_replace ='(\w+\s?[-]?\w+)\s-', value = '', regex = True)

In [215]:
pd.DataFrame(data)

,구분,Jan.,Feb.,Mar.,Apr.,May,Jun.,Jul.,Aug.,Sep.,Oct.,Nov.,Dec.,Total,sales_type,age,year,brand,model
0,Audi - Q2 35 TDI,1,0,0,0,0,0,0,0,0,0,0,0,1,개인,~19,2016,Audi,Q2 35 TDI
1,BMW - 218d Gran Coupe,0,1,0,0,0,0,0,0,0,0,0,0,1,개인,~19,2016,BMW,218d Gran Coupe
2,BMW - 520,1,0,0,0,0,0,0,0,0,0,0,0,1,개인,~19,2016,BMW,520
3,Bentley - Continental GT V8,0,1,0,0,0,0,0,0,0,0,0,0,1,개인,~19,2016,Bentley,Continental GT V8
4,Honda - CR-V Hybrid,0,0,0,1,0,0,0,0,0,0,0,0,1,개인,~19,2016,Honda,CR-V Hybrid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45907,Volvo - XC60 B5 AWD,0,0,0,28,0,0,0,0,0,0,0,0,28,법인,70~,2021,Volvo,XC60 B5 AWD
45908,Volvo - XC60 B6 AWD,0,0,9,16,0,0,0,0,0,0,0,0,25,법인,70~,2021,Volvo,XC60 B6 AWD
45909,Volvo - XC60 T8 AWD,78,112,45,16,0,0,0,0,0,0,0,0,251,법인,70~,2021,Volvo,XC60 T8 AWD
45910,Volvo - XC90 B6 AWD,0,0,20,22,0,0,0,0,0,0,0,0,42,법인,70~,2021,Volvo,XC90 B6 AWD


In [ ]:
data.columns = ["구분", "1월", "2월", "3월", "4월", "5월", "6월", "7월", "8월", "9월", "10월", "11월", "12월", "total", "sales_type", "age", "year", "brand","model"]
data.head()

In [221]:
data = pd.melt(data, id_vars=['brand', 'model', 'sales_type', 'age', 'year'], 
               value_vars=['1월', '2월','3월','4월','5월','6월','7월','8월','9월','10월','11월','12월'], var_name='month')
data.head()

KeyError: "The following 'value_vars' are not present in the DataFrame: ['10월', '11월', '12월', '1월', '2월', '3월', '4월', '5월', '6월', '7월', '8월', '9월']"

In [218]:
data = data[["brand", "model", "value", "sales_type", "age", "month", "year"]]

In [219]:
data.to_csv('specified_model.csv', index=False, encoding='utf-8-sig')
data.to_excel('specified_model.xlsx')